## 1. 基本操作

In [2]:
# 导入模块
from wxpy import *


# 初始化机器人，扫码登陆
bot = Bot()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as Jlan


### 搜索好友

In [12]:
my_friends = bot.friends()
lan = my_friends.search('穿林打叶')[0]

### 发送消息

In [13]:
lan.send('神经病')

↪ 穿林打叶 : 神经病 (Text)

### 自动响应消息

In [35]:
# 打印来自其他好友、群聊和公众号的消息
@bot.register()
def print_others(msg):
    print(msg)

In [40]:
# 回复 my_friend 的消息 (优先匹配后注册的函数!)
@bot.register(lan)
def reply_my_friend(msg):
    return 'received: {} ({})'.format(msg.text, msg.type)

In [29]:
# 自动接受新的好友请求
@bot.register(msg_types=FRIENDS)
def auto_accept_friends(msg):
    # 接受好友请求
    new_friend = msg.card.accept()
    # 向新的好友发送消息
    new_friend.send('哈哈，我自动接受了你的好友请求')

澎湃新闻 : 又一名高铁保洁员确诊，曾搭乘这15趟列车 (Sharing)


### 群操作相关

In [36]:
# 查找群
my_groups = bot.groups()
# my_groups
wxpy_test_group = my_groups.search('wxpy测试')[1]
wxpy_test_group

<Group: wxpy测试>

In [17]:
# 发送群消息
wxpy_test_group.send('hi')

↪ wxpy测试群 : hi (Text)

In [18]:
# 输出群成员
wxpy_test_group.members

[<Member: 兰儿>, <Member: 穿林打叶>, <Member: Jlan>]

In [19]:
# 测试好友是否在群中
lan in wxpy_test_group

True

## 2. 小实战
检测指定的微信群，当有人@时，能自动回复

In [3]:
# 智能对话机器人接口

import urllib
import base64
import requests
import json

chatbot_url = 'http://irobot.percent.cn/api/chat'

def chatbot_api(input_text):
    content_after_quote = urllib.parse.quote(input_text)
    content_after_b64encode = base64.b64encode(content_after_quote.encode())  #
    content = content_after_b64encode[::-1]  # 字符串反转得到
    post_data = {'session_id': '1111', 'content': content.decode()}
    reply = requests.post(chatbot_url, data=json.dumps(post_data))
    reply = reply.json()["content"]
    return reply

In [4]:
chatbot_api('你好')

'你终于来和我说话啦'

In [15]:
my_group = bot.groups().search('吃饭')[0]
# my_friend = my_group.search('穿林打叶')[0]

print(my_group)

<Group: 工作、吃饭订餐群>


In [16]:
# 回复指定群聊中被 @ 的消息
import jieba

kws = ['傻逼', 'sb']

@bot.register(my_group, TEXT)
def auto_reply(msg):
    # 如果是群聊，但没有被 @，则不回复
    print(f'input msg: {msg.text}')
    if isinstance(msg.chat, Group):
        for kw in kws:
            if kw in msg.text:
                return '请文明讲话'
        return ''.join(jieba.lcut(msg.text)[-3:])
    if msg.is_at:
        # 回复消息内容和类型
#         return '收到消息: {} ({})'.format(msg.text, msg.type)
        reply = chatbot_api(msg.text.split()[-1])
        print(f'reply: {reply}')
        return reply

input msg: 那就
input msg: 我去
input msg: 今天好像小区通知从今天开始，外地到京的要自行隔离14天
